# Comparing CPAC_N10_11_10_20 dataset with Compiled_Yinsole

- **Model**: Gradient Boosted Trees (histogram-based)
- **Target(s)**: `TF_Pelvis_Moment_X_BWBH`, `TF_Pelvis_Moment_Y_BWBH`
- **Features**: trunk IMU
- **Participants: 2,4,5,8,10
- **Results**: 
  - $r^2$ scores (by cross-validation)
  - feature importances (permutation-based, using the full dataset for training)
  - predictions (merged, by cross-validation)
- **Evaluation strategy**: cross-validation (leave one subject out)

## Libraries

In [1]:
# Standard library
import warnings
import os


# Third party
import numpy as np
import pandas as pd
import sklearn
assert sklearn.__version__ >= "0.21", "Use the conda_python3_latest kernel!"
from sklearn.experimental import enable_hist_gradient_boosting  # noqa
from sklearn import (ensemble, metrics, preprocessing, 
                     pipeline, inspection, model_selection)

from IPython.display import display, Markdown


# Local
import utils

## Load Dataset

In [2]:
DATASET = "CPAC_N10_11_10_20"
DATASET_CSV = f"s3://cpac/ORIG/{DATASET}/CPAC10S_N10_11_10_20.csv"
DATASET_README = f"s3://cpac/ORIG/{DATASET}/READ_ME.xlsx"
RESULTS_DIR = f"results/{DATASET}"


df_orig = utils.load_dataset(DATASET_CSV)
df_orig.describe()

,M_Trial_Num,M_Mass,M_Mass_to_L5S1,M_sub_task_indices,M_sub_task_num,M_include_overall,M_Index,M_Sub,M_sub_task_num_overall,M_Index_overall,...,RWEO_01_02_00_00_INSOLE_RX_ML_threshF50_mm,RWEO_01_02_00_00_INSOLE_RY_AP_threshF50_mm,RWEF_03_00_00_00_INSOLE_LFORCE_threshF50_BW,RWEF_03_04_00_00_INSOLE_LX_ML_threshF50_BH,RWEF_03_04_00_00_INSOLE_LY_AP_threshF50_BH,RWEF_01_00_00_00_INSOLE_RFORCE_threshF50_BW,RWEF_01_02_00_00_INSOLE_RX_ML_threshF50_BH,RWEF_01_02_00_00_INSOLE_RY_AP_threshF50_BH,RWSF_SCALED_RINSOLE_BW,RWSF_SCALED_LINSOLE_BW
count,1.971017e+06,1.971017e+06,567.0,0.0,0.0,1.971017e+06,1.971017e+06,1.971017e+06,1971017.0,1.971017e+06,...,1.683991e+06,1.683991e+06,1.967609e+06,1.661377e+06,1.661377e+06,1.967609e+06,1.683991e+06,1.683991e+06,1.967609e+06,1.967609e+06
mean,6.942408e+01,1.053204e+01,0.0,NaN,NaN,8.154633e-01,2.716049e+03,5.396072e+00,0.0,9.236236e+04,...,4.899182e+01,1.319880e+02,4.390082e-01,2.785011e-02,6.597227e-02,4.570948e-01,2.698083e-02,7.262811e-02,4.262551e-01,3.744147e-01
std,2.367938e+01,5.845028e+00,0.0,NaN,NaN,3.879214e-01,2.566750e+03,2.934289e+00,0.0,6.388543e+04,...,9.183171e+00,5.335901e+01,3.258924e-01,5.077019e-03,3.046704e-02,3.370890e-01,5.108799e-03,2.956883e-02,3.576745e-01,3.189683e-01
min,1.000000e+00,0.000000e+00,0.0,NaN,NaN,0.000000e+00,1.000000e+00,1.000000e+00,0.0,0.000000e+00,...,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,-5.407405e-02,-5.624677e-02
25%,4.900000e+01,5.000000e+00,0.0,NaN,NaN,1.000000e+00,5.980000e+02,3.000000e+00,0.0,3.568000e+04,...,4.514000e+01,8.828000e+01,1.586822e-01,2.579404e-02,3.957558e-02,1.723314e-01,2.458140e-02,4.846361e-02,1.368566e-01,1.042724e-01
50%,7.900000e+01,1.000000e+01,0.0,NaN,NaN,1.000000e+00,1.875000e+03,5.000000e+00,0.0,9.043000e+04,...,5.076000e+01,1.302100e+02,4.173078e-01,2.872237e-02,6.118229e-02,4.304055e-01,2.806366e-02,7.106925e-02,3.676134e-01,3.228916e-01
75%,8.700000e+01,1.500000e+01,0.0,NaN,NaN,1.000000e+00,4.215000e+03,8.000000e+00,0.0,1.451810e+05,...,5.473000e+01,1.777800e+02,6.735348e-01,3.100000e-02,9.114589e-02,6.958240e-01,3.011671e-02,9.697091e-02,6.673267e-01,6.007689e-01
max,9.600000e+01,2.300000e+01,0.0,NaN,NaN,1.000000e+00,1.411700e+04,1.000000e+01,0.0,2.368970e+05,...,7.911000e+01,2.677800e+02,2.068044e+00,4.425281e-02,1.506798e-01,2.041016e+00,4.326744e-02,1.504382e-01,1.981410e+00,1.950301e+00


## Associate column names

In [3]:
def _get_columns_with_prefix(df, prefix):
    columns = []
    for column in df.columns:
        if column.startswith(prefix):
            columns.append(column)
    return columns
    
def get_target_names(df):
    return _get_columns_with_prefix(df, "T_")

def get_meta_names(df):
    return _get_columns_with_prefix(df, "M_")    

## Clean-up dataset

- Remove samples based on `M_include_overall`

In [4]:
df = df_orig[df_orig["M_include_overall"] > 0]

# Weed out wonky subjects
#df = df[df["M_Sub"].isin([2,4,5,6,7,8,9])]
#RESULTS_DIR += "_nowonky"

# Select subseet of participants
df = df[df["M_Sub"].isin([2,4,5,8,10])]

print(f"Number of samples: {df.shape[0]:,d} (before clean-up: {df_orig.shape[0]:,d})")
print(f"Number of trials: {len(df['M_Trial_Name'].unique())} (before clean-up: {len(df_orig['M_Trial_Name'].unique())})")
print(f"Number of subjects: {len(df['M_Sub'].unique())}")

Number of samples: 851,269 (before clean-up: 1,971,017)
Number of trials: 124 (before clean-up: 174)
Number of subjects: 5


## Predictor configurations (recipes)

In [5]:
def predictor_short_name(predictor):
    return predictor[17:]

def predictor_sensor_number(predictor):
    #return int(predictor[5:7])
    return predictor[5:7]

def filter_predictors(all_predictors, patterns):
    if isinstance(patterns, str):
        patterns = (patterns,)
        
    predictors = []
    for predictor in all_predictors:
        for pattern in patterns:
            if pattern in predictor:
                predictors.append(predictor)
                break
    return predictors



# def build_feature_sets(df):
#     readme_xls = utils.download_dataset(DATASET_README)
#     readme = pd.read_excel(readme_xls, sheet_name="Recipe_FINAL")
    
#     feature_sets = {}
    
#     recipes = readme.iteritems()
#     next(recipes)   # first column is bogus
#     for recipe_num, recipe in recipes:
#         recipe_desc = recipe[3]
#         recipe_filter_1 = [filter for filter in (recipe[7], recipe[9]) if isinstance(filter, str)]
#         recipe_filter_2 = [filter for filter in recipe[11:] if isinstance(filter, str)]
#         recipe_name = f"Recipe {recipe_num}: {recipe_desc}"
#         feature_sets[recipe_name] = filter_predictors(filter_predictors(df.columns, recipe_filter_1), recipe_filter_2)
    
#     return feature_sets

# feature_sets = build_feature_sets(df)


feature_sets = {
    "trunk IMU": [
       "RWRF_05_12_00_00_L5S1_jointAngle_X_LB",
        "RWRF_05_12_00_00_L5S1_jointAngle_Y_RO",
        "RWRF_05_12_00_00_L5S1_jointAngle_Z_FE",
        "RWRF_05_12_00_00_L4L3_jointAngle_X_LB",
        "RWRF_05_12_00_00_L4L3_jointAngle_Y_RO",
        "RWRF_05_12_00_00_L4L3_jointAngle_Z_FE",
        "RWRF_05_12_00_00_L1T12_jointAngle_X_LB",
        "RWRF_05_12_00_00_L1T12_jointAngle_Y_RO",
        "RWRF_05_12_00_00_T9T8_jointAngle_X_LB",
        "RWRF_05_12_00_00_L1T12_jointAngle_Z_FE",
        "RWRF_05_12_00_00_T9T8_jointAngle_Y_RO",
        "RWRF_05_12_00_00_T9T8_jointAngle_Z_FE",
        
        "RWRF_12_00_00_00_L5_orientation_q1",
        "RWRF_12_00_00_00_L5_orientation_q2",
        "RWRF_12_00_00_00_L5_orientation_q3",
        "RWRF_12_00_00_00_L5_orientation_q4",
        "RWRF_12_00_00_00_L5_orientation_eZ_tra",
        "RWRF_12_00_00_00_L5_orientation_eY_sag",
        "RWRF_12_00_00_00_L5_orientation_eX_fro",
        "RWRF_12_00_00_00_L5_velocity_X_for",
        "RWRF_12_00_00_00_L5_velocity_Y_sid",
        "RWRF_12_00_00_00_L5_velocity_Z_ver",
        "RWRF_12_00_00_00_L5_acceleration_X_for",
        "RWRF_12_00_00_00_L5_acceleration_Y_sid",
        "RWRF_12_00_00_00_L5_acceleration_Z_ver",

        "RWRF_12_00_00_00_L3_orientation_q1",
        "RWRF_12_00_00_00_L3_orientation_q2",
        "RWRF_12_00_00_00_L3_orientation_q3",
        "RWRF_12_00_00_00_L3_orientation_q4",
        "RWRF_12_00_00_00_L3_orientation_eZ_tra",
        "RWRF_12_00_00_00_L3_orientation_eY_sag",
        "RWRF_12_00_00_00_L3_orientation_eX_fro",
        "RWRF_12_00_00_00_L3_velocity_X_for",
        "RWRF_12_00_00_00_L3_velocity_Y_sid",
        "RWRF_12_00_00_00_L3_velocity_Z_ver",
        "RWRF_12_00_00_00_L3_acceleration_X_for",
        "RWRF_12_00_00_00_L3_acceleration_Y_sid",
        "RWRF_12_00_00_00_L3_acceleration_Z_ver",

        "RWRF_12_00_00_00_T12_orientation_q1",
        "RWRF_12_00_00_00_T12_orientation_q2",
        "RWRF_12_00_00_00_T12_orientation_q3",
        "RWRF_12_00_00_00_T12_orientation_q4",
        "RWRF_12_00_00_00_T12_orientation_eZ_tra",
        "RWRF_12_00_00_00_T12_orientation_eY_sag",
        "RWRF_12_00_00_00_T12_orientation_eX_fro",
        "RWRF_12_00_00_00_T12_velocity_X_for",
        "RWRF_12_00_00_00_T12_velocity_Y_sid",
        "RWRF_12_00_00_00_T12_velocity_Z_ver",
        "RWRF_12_00_00_00_T12_acceleration_X_for",
        "RWRF_12_00_00_00_T12_acceleration_Y_sid",
        "RWRF_12_00_00_00_T12_acceleration_Z_ver",
        
        #"RWRF_12_00_00_00_T8_orientation_q1",
        #"RWRF_12_00_00_00_T8_orientation_q2",
        #"RWRF_12_00_00_00_T8_orientation_q3",
        #"RWRF_12_00_00_00_T8_orientation_q4",
        #"RWRF_12_00_00_00_T8_orientation_eZ_tra",
        #"RWRF_12_00_00_00_T8_orientation_eY_sag",
        #"RWRF_12_00_00_00_T8_orientation_eX_fro",
        #"RWRF_12_00_00_00_T8_velocity_X_for",
        #"RWRF_12_00_00_00_T8_velocity_Y_sid",
        #"RWRF_12_00_00_00_T8_velocity_Z_ver",
        #"RWRF_12_00_00_00_T8_acceleration_X_for",
        #"RWRF_12_00_00_00_T8_acceleration_Y_sid",
        #"RWRF_12_00_00_00_T8_acceleration_Z_ver",
    ]
}

for feature_set_name, predictors in feature_sets.items():
    sensors = set(map(predictor_sensor_number, predictors))
    print(f"{feature_set_name}\n\tPredictors: {len(predictors)}, Sensors: {len(sensors)}\n")

trunk IMU
	Predictors: 51, Sensors: 2



## Train and evaluate boosted tree models

In [6]:
def evaluate(target_name, feature_names):
    X, y, groups = df[feature_names], df[target_name], df["M_Sub"]
    
    model = pipeline.Pipeline([
        ('scaler', preprocessing.StandardScaler()),
        ('gboost', ensemble.HistGradientBoostingRegressor())
    ])
    
    logo = model_selection.LeaveOneGroupOut()

    prediction = model_selection.cross_val_predict(
        model, X, y, cv=logo, groups=groups, n_jobs=-1)

    r2_score = {}
    for idx_train, idx_test in logo.split(df, groups=groups):
        subject = df.iloc[idx_test[0]]["M_Sub"]
        r2_score[subject] = metrics.r2_score(y.iloc[idx_test], prediction[idx_test])
        
    r2_score = pd.Series(r2_score)
    prediction = pd.Series(prediction, index=y.index)
    
    # Feature importances on the full training set
    model.fit(X, y)
    perm_imp = inspection.permutation_importance(model, X, y, n_repeats=5, n_jobs=10)
    importance = pd.Series(perm_imp.importances_mean, index=X.columns)
    importance.sort_values(ascending=False, inplace=True)

    return r2_score, importance, prediction, y

## Run experiments, save data

In [13]:
os.makedirs(RESULTS_DIR, exist_ok=True)


target_name_x = "TF_Pelvis_Moment_X_BWBH"
target_name_y = "TF_Pelvis_Moment_Y_BWBH"


r2_mean_scores_x = {}
r2_mean_scores_y = {}


for feature_set_name, feature_names in feature_sets.items():
    r2_score_x, importance_x, prediction_x, target_x = evaluate(target_name_x, feature_names)
    r2_mean_scores_x[feature_set_name] = r2_score_x.mean()
    r2_score_y, importance_y, prediction_y, target_y = evaluate(target_name_y, feature_names)
    r2_mean_scores_y[feature_set_name] = r2_score_y.mean()
    display(
        Markdown(
            "---\n"
            f"**Features**: {feature_set_name}  \n"
            f"**$R^2$ ({target_name_x}) = {r2_mean_scores_x[feature_set_name]:.3f}**\n"
            f"**$R^2$ ({target_name_y}) = {r2_mean_scores_y[feature_set_name]:.3f}**\n"
        )
    )

    with pd.ExcelWriter(f"{RESULTS_DIR}/R2_scores.xlsx") as writer:
        df_results = pd.DataFrame({f"R2 - {target_name_x}": r2_mean_scores_x, 
                                   f"R2 - {target_name_y}": r2_mean_scores_y,})
        df_results.to_excel(writer, sheet_name="R2 Scores")


    short_name = feature_set_name.split(":")[0].replace(" ", "_")
    with pd.ExcelWriter(f"{RESULTS_DIR}/{short_name}_results.xlsx") as writer:

        df_results = pd.DataFrame({f"R2 - {target_name_x}": r2_score_x,
                                   f"R2 - {target_name_y}": r2_score_y})
        df_results.to_excel(writer, index_label="Test Subject", sheet_name="R2 Scores")


        df_results = pd.DataFrame(
            {
                #"Short name": map(predictor_short_name, importance_x.index),
                f"Importance - {target_name_x}": importance_x,
                f"Importance - {target_name_y}": importance_y,
            }
        )
        df_results.to_excel(writer, sheet_name="Importance")
        
    df_results = pd.DataFrame(
        {
            f"Predictions - {target_name_x}": prediction_x,
            f"Target - {target_name_x}": target_x,
            f"Predictions - {target_name_y}": prediction_y,
            f"Target - {target_name_y}": target_y
        }
    )
    df_results.to_csv(f"{RESULTS_DIR}/{short_name}_predictions.csv")



---
**Features**: trunk IMU  
**$R^2$ (TF_Pelvis_Moment_X_BWBH) = 0.793**
**$R^2$ (TF_Pelvis_Moment_Y_BWBH) = 0.654**
